## [Description](#Description_)
## [Research](#Research_)
## [Imports](#Imports_)
## [Globals](#Globals_)
## [Utils](#Utils_)
## [Setup](#Setup_)
## [Data](#Data_)
## [Data exploration](#Data_exploration_)
## [Model](#Model_)
## [Training](#Training_)
## [Results](#Results_)

## Description <span id=Description_></span>

# План домашней работы
1. Реализовать алгоритмы **item2item**, **ALS**, **IALS** (2 балл за каждый)
2. Посчитать метрику предсказаний **MRR@100** выбрасывая случайный лайк пользователя (2 балла)

Будем решать задачу предсказания: на 4/5 пользователей учимся, на 1/5 выбрасываем случайный лайк и пытаемся предсказать его беря топ 100 наших лучших предсказаний для этого пользователя.

MRR@100 будет равно $1/(p+1)$, где $p$ - позиция на которой оказался выброшенный лайк в нашем ранжировании и 0 если в топ 100 его не было.

3. Подобрать параметры алгоритмов для максимизации MRR@100 (1 балл)
4. Сравнить похожести айтемов получаюшиеся для item2item, ALS, IALS (1 балл)

Замерить насколько получаются похожими топы похожестей. Так же рекомендуется взять 5-топовых (или любимых) треков и посмотреть на похожести которые получаются для них в разных алгоритмах.

### Ссылки на датасет музыки:

1. Матричка: https://disk.yandex.ru/d/cLbDbw3mCido_w
2. Имена тайтемов: https://disk.yandex.ru/d/pPCaGJOqcpcABw
3. Линки между айтемами: https://disk.yandex.ru/d/hMErnDJqtVm9HQ

## Research <span id=Research_></span>

- [Implicit ALS](http://yifanhu.net/PUB/cf.pdf)

## Imports <span id=Imports_></span>

In [44]:
import glob
import numpy as np
import os
import pandas as pd
import requests
import scipy
from tqdm.autonotebook import tqdm
import types
import urllib
import zipfile

%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

## Globals <span id=Globals_></span>

In [12]:
CONFIG = types.SimpleNamespace()
DATA = types.SimpleNamespace()

<IPython.core.display.Javascript object>

## Utils <span id=Utils_></span>

### Markdown

In [ ]:
def make_new_markdown_section_with_link(section, header="##", do_print=True):
    section_id = section.replace(" ", "_") + "_"
    section_link = f"{header} [{section}](#{section_id})"
    section_header = f"{header} {section} <span id={section_id}></span>"
    if do_print:
        print(section_link + "\n" + section_header)
    return section_link, section_header


def make_several_sections(
    section_names=(
        "Description",
        "Imports",
        "Globals",
        "Setup",
        "Data",
        "Data exploration",
        "Model",
        "Training",
        "Results",
    )
):
    links, headers = zip(
        *[
            make_new_markdown_section_with_link(sn, do_print=False)
            for sn in section_names
        ]
    )
    print("\n".join(links + ("",) + headers))

### Yadisk

In [7]:
def get_yadisk_download_url(
    yadisk_url: str,
    base_url="https://cloud-api.yandex.net/v1/disk/public/resources/download?",
) -> str:
    final_url = base_url + urllib.parse.urlencode(dict(public_key=yadisk_url))
    response = requests.get(final_url)
    download_url = response.json()["href"]
    return download_url


def write_response_content(url: str, filename: str) -> None:
    download_response = requests.get(url)
    with open(filename, "wb") as f:
        f.write(download_response.content)

<IPython.core.display.Javascript object>

### Cli

In [42]:
def unzip(zip_path, save_path=None, delete_zip=False):
    !unzip {zip_path} {"-d "+ save_path if save_path else ""}
    if delete_zip:
        for path in glob.glob(zip_path):
            if path.endswith(".zip"):
                !trash {path}

<IPython.core.display.Javascript object>

## Setup <span id=Setup_></span>

In [46]:
DATA.yadisk_df_url = "https://disk.yandex.ru/d/cLbDbw3mCido_w"
DATA.yadisk_names_url = "https://disk.yandex.ru/d/pPCaGJOqcpcABw"
DATA.yadisk_links_url = "https://disk.yandex.ru/d/hMErnDJqtVm9HQ"

DATA.df_filename = "data/music/df"
DATA.names_filename = "data/music/names"
DATA.links_filename = "data/music/links"

<IPython.core.display.Javascript object>

In [49]:
for url, filename in tqdm(
    zip(
        [DATA.yadisk_df_url, DATA.yadisk_names_url, DATA.yadisk_links_url],
        [DATA.df_filename, DATA.names_filename, DATA.links_filename],
    )
):
    if not os.path.exists(filename):
        zip_filename = filename + ".zip"
        write_response_content(
            url=get_yadisk_download_url(yadisk_url=url),
            filename=zip_filename,
        )
        unzip(zip_path=zip_filename, save_path=filename, delete_zip=True)

0it [00:00, ?it/s]

<IPython.core.display.Javascript object>

In [50]:
DATA.df = pd.read_json(DATA.df_filename, lines=True)

<IPython.core.display.Javascript object>

In [53]:
DATA.df.head()

,user_id,track_ids
0,0,"[99262, 589498, 590262, 590303, 590692, 590803..."
1,1,"[24417, 108208, 108209, 592642, 628687, 733449..."
2,2,"[264937, 672689, 4321285, 5335351, 5658525, 58..."
3,3,"[6006252, 21642261, 21642265, 24692821, 259952..."
4,4,"[966, 4094, 9760, 9769, 18392, 19042, 21184, 2..."


<IPython.core.display.Javascript object>

## Data <span id=Data_></span>

In [80]:
def build_sparse_coo_matrix(df, dtype=np.int32):
    row_indices = []
    col_indices = []

    for i, row in tqdm(df.iterrows(), total=len(df)):
        user_id = row["user_id"]
        item_ids = row["track_ids"]
        row_indices += [user_id] * len(item_ids)
        col_indices += item_ids

    data = np.ones_like(row_indices)
    coo_matrix = scipy.sparse.coo_matrix(
        (data, (row_indices, col_indices)), dtype=dtype
    )
    return coo_matrix

<IPython.core.display.Javascript object>

In [ ]:
DATA.coo_matrix = build_sparse_coo_matrix(DATA.df)

  0%|          | 0/1373221 [00:00<?, ?it/s]

In [82]:
DATA.coo_matrix.dtype

dtype('int32')

<IPython.core.display.Javascript object>

In [ ]:
data = 
row_indices = 
col_indices = 


In [ ]:
total_interactions_count = dataset.track_ids.map(len).sum()
user_coo, item_coo = np.zeros(total_interactions_count, dtype=np.int32), np.zeros(total_interactions_count, dtype=np.int32)
pos = 0
for u, tracks in enumerate(tqdm(dataset.track_ids)):
    user_coo[pos:pos+len(tracks)] = u
    item_coo[pos:pos+len(tracks)] = tracks
    pos += len(tracks)
shape = (max(user_coo) + 1, max(item_coo) + 1)
user_item_matrix = sp.coo_matrix((np.ones(len(user_coo)), (user_coo, item_coo)), shape=shape)
user_item_matrix = user_item_matrix.tocsr()
sp.save_npz("data_train.npz", user_item_matrix)
# Cleanup memory. Later you need just data_train.npz
del user_coo
del item_coo
del dataset

In [ ]:
user_item_matrix = sp.load_npz("data_train.npz")

In [ ]:
item_weights = np.array(user_item_matrix.tocsc().sum(0))[0]
top_to_bottom_order = np.argsort(-item_weights)
item_mapping = np.empty(top_to_bottom_order.shape, dtype=int)
item_mapping[top_to_bottom_order] = np.arange(len(top_to_bottom_order))
total_item_count = (item_weights > 0).sum()
total_user_count = user_item_matrix.shape[0]

def build_dataset(user_item_matrix, item_pct, user_pct):
    user_count, item_count = int(total_user_count * user_pct), int(total_item_count * item_pct)
    item_ids = top_to_bottom_order[:item_count]
    user_ids = np.random.choice(np.arange(user_item_matrix.shape[0]), size=user_count, replace=False)
    train = user_item_matrix[user_ids]
    train = train[:, item_ids]
    return train

In [ ]:
small_dataset = build_dataset(user_item_matrix, 0.05, 0.05)

## Data exploration <span id=Data_exploration_></span>
## Model <span id=Model_></span>
## Training <span id=Training_></span>
## Results <span id=Results_></span>